# Deep Neural Network

## Description

In this notebook, we will try to build a deep neural network (multiple hidden layers), and then use it to classify the cat vs non-cat images.

## Import Libraries

In [2]:
import numpy as np
import h5py

import sys
sys.path.append('../utils/')
from nn_utils import *
from data_utils import *

np.random.seed(1)

## Build The Neural Network

In [6]:
# helper functions

# 1- parameters initialization
def initialize_parameters(dimensions):
    np.random.seed(1)
    parameters = {}
    L = len(dimensions)
    for l in range(1, L):
        parameters[f'W{l}'] = np.random.randn(dimensions[l], dimensions[l-1]) / np.sqrt(dimensions[l-1])
        parameters[f'b{l}'] = np.zeros((dimensions[l], 1))
    return parameters

def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    return Z, cache

def linear_activation_forward(A_prev, W, b, activation):
    Z, linear_cache = linear_forward(A_prev, W, b)
    if activation == 'sigmoid':
        A, activation_cache = sigmoid(Z)
    elif activation == 'relu':
        A, activation_cache = relu(Z)
    cache = (linear_cache, activation_cache)
    return A, cache

def forward_prop(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters[f'W{l}'], parameters[f'b{l}'], 'relu')
        caches.append(cache)
    AL, cache = linear_activation_forward(A, parameters[f'W{L}'], parameters[f'b{L}'], 'sigmoid')
    caches.append(cache)
    return AL, caches

def compute_cost(AL, Y):
    size = Y.shape[1]
    cost = (1/size) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
    cost = np.squeeze(cost)
    return cost

def linear_backward(dZ, cache):
    A_prev, W, b = cache
    size = A_prev.shape[1]
    dW = 1/size * np.dot(dZ, A_prev.T)
    db = 1/size * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    return dA_prev, dW, db

def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == 'relu':
        dZ = relu_backward(dA, activation_cache)
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dA, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

def backward_prop(AL, Y, caches):
    grads = {}
    L = len(caches)
    Y = Y.reshape(AL.shape)
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    current_cache = caches[L-1]
    grads[f'dA{L-1}'], grads[f'dW{L}'], grads[f'db{L}'] = linear_activation_backward(dAL, current_cache, 'sigmoid')
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads[f'dA{l+1}'], current_cache, 'relu')
        grads[f'dA{l}'] = dA_prev_temp
        grads[f'dW{l+1}'] = dW_temp
        grads[f'db{l+1}'] = db_temp
    return grads

def update_parameters(params, grads, learning_rate):
    parameters = params.copy()
    L = len(parameters) // 2
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
    return parameters

def model(X, Y, dimensions, learning_rate=0.0075, num_iterations=3000, print_cost=False):
    costs = []
    parameters = initialize_parameters(dimensions)
    for i in range(num_iterations):
        AL, caches = forward_prop(X, parameters)
        cost = compute_cost(AL, Y)
        grads = backward_prop(AL, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate)
        if print_cost and i % 100 == 0 or i == num_iterations-1:
            print('Cost after iteration %i: %f'%(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    return parameters, costs    

def predict(X, Y, parameters):
    size = X.shape[1]
    n = len(parameters)
    p = np.zeros((1, size))
    probas, caches = forward_prop(X, parameters)
    for i in range(probas.shape[1]):
        if probas[0, i] > 0.5:
            p[0, i] = 1
        else:
            p[0, i] = 0
    print("Accuracy: "  + str(np.sum((p == Y)/size)))
    return p

## Load and Preprocess Data

In [4]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()
train_x, test_x = preprocess_data(train_x_orig, test_x_orig)

## Test The Neural Network

In [7]:
dimensions = [12288, 20, 7, 5, 1]
parameters, costs = model(train_x, train_y, dimensions, learning_rate=0.01, num_iterations=2500, print_cost=True)

Cost after iteration 0: 0.771749
Cost after iteration 100: 0.669270
Cost after iteration 200: 0.638874
Cost after iteration 300: 0.597884
Cost after iteration 400: 0.568827
Cost after iteration 500: 0.461260
Cost after iteration 600: 0.508484
Cost after iteration 700: 0.327596
Cost after iteration 800: 0.310398
Cost after iteration 900: 0.248831
Cost after iteration 1000: 0.207309
Cost after iteration 1100: 0.140485
Cost after iteration 1200: 0.115670
Cost after iteration 1300: 0.099260
Cost after iteration 1400: 0.085845
Cost after iteration 1500: 0.074975
Cost after iteration 1600: 0.067809
Cost after iteration 1700: 0.058402
Cost after iteration 1800: 0.052054
Cost after iteration 1900: 0.047680
Cost after iteration 2000: 0.042259
Cost after iteration 2100: 0.037797
Cost after iteration 2200: 0.034730
Cost after iteration 2300: 0.031391
Cost after iteration 2400: 0.028788
Cost after iteration 2499: 0.026663


In [8]:
train_preds = predict(train_x, train_y, parameters)

Accuracy: 0.9999999999999998


In [9]:
test_preds = predict(test_x, test_y, parameters)

Accuracy: 0.8400000000000001
